In [28]:
from cloudvolume import CloudVolume
import numpy as np
import cc3d
import cv2


### Preparation - ensure a volume with `tissue=True`, `background=False`

In [122]:
# Current img mask uses 0 for tissue and 1 for background, but also contains 0 for missing / outerior data
cv = CloudVolume("precomputed://gs://dkronauer-ant-001-alignment-final/img_mask", mip=[512,512,42], bounded=False, fill_missing=True, background_color=0)
bounds = cv.bounds
data = cv[bounds.grow(1).to_slices()][:,:,1:-1]

Downloading: 100%|██████████| 6112/6112 [01:12<00:00, 84.54it/s] 


In [123]:
# Fill outer space for each individual section with background color
for i in range(data.shape[2]):
  print(f"Processing section {i}")
  section = data[:,:,i].squeeze() == 0  # 0 is tissue!
  section_cc = cc3d.connected_components(section, connectivity=4)
  section[section_cc == 1] = 0  # ID 1 will be the padded background
  data[:,:,i] = section[:,:,np.newaxis]

Processing section 0
Processing section 1
Processing section 2
Processing section 3
Processing section 4
Processing section 5
Processing section 6
Processing section 7
Processing section 8
Processing section 9
Processing section 10
Processing section 11
Processing section 12
Processing section 13
Processing section 14
Processing section 15
Processing section 16
Processing section 17
Processing section 18
Processing section 19
Processing section 20
Processing section 21
Processing section 22
Processing section 23
Processing section 24
Processing section 25
Processing section 26
Processing section 27
Processing section 28
Processing section 29
Processing section 30
Processing section 31
Processing section 32
Processing section 33
Processing section 34
Processing section 35
Processing section 36
Processing section 37
Processing section 38
Processing section 39
Processing section 40
Processing section 41
Processing section 42
Processing section 43
Processing section 44
Processing section 4

### Downsample / Close Holes

In [195]:
# "Downsample" in z - that should also fill gaps caused by missing sections
grouped_data = np.mean(data.astype(np.float32).reshape(data.shape[0], data.shape[1], -1, 8), axis=3) > 2.0/8.0

In [196]:
# First pass on removing holes
# Remove every component of tissue that is not part of the main volume
grouped_data = cc3d.dust(grouped_data, threshold=800*752*4, connectivity=6)

# Fill in holes within the tissue
grouped_data = ~cc3d.dust(~grouped_data, threshold=800*752*4, connectivity=6)

In [197]:
# For good measure, also close a bit along z
open_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
close_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
for i in range(grouped_data.shape[0]):
    section = grouped_data[i,:,:]
    opened_cross_section = cv2.morphologyEx(section.astype(np.float32), cv2.MORPH_OPEN, open_kernel)
    closed_cross_section = cv2.morphologyEx(opened_cross_section.astype(np.float32), cv2.MORPH_CLOSE, close_kernel)
    grouped_data[i,:,:] = closed_cross_section.astype(bool)

In [198]:
# Some additional closing and opening to smoothen the edges within each section
open_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
close_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
for i in range(grouped_data.shape[2]):
    section = grouped_data[:,:,i]
    opened_section = cv2.morphologyEx(section.astype(np.float32), cv2.MORPH_OPEN, open_kernel)
    closed_section = cv2.morphologyEx(opened_section.astype(np.float32), cv2.MORPH_CLOSE, close_kernel)
    grouped_data[:,:,i] = closed_section.astype(bool)

In [199]:
# Second pass on removing holes
# Remove every component of tissue that is not part of the main volume
grouped_data = cc3d.dust(grouped_data, threshold=800*752*4, connectivity=6)

# Fill in holes within the tissue
grouped_data = ~cc3d.dust(~grouped_data, threshold=800*752*4, connectivity=6)

In [254]:
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = 'segmentation',
    data_type = 'uint8',
    encoding = 'raw',
    resolution = [ 512, 512, 336 ], # X,Y,Z values in nanometers, 40nm in each dim
    voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
    chunk_size = [ 800, 752, 4 ], # rechunk of image X,Y,Z in voxels
    volume_size = [ 800, 752, 764], # X,Y,Z size in voxels
)

cv_up = CloudVolume("precomputed://gs://dkronauer-ant-001-alignment-final/tissue_mesh_mask", mip=0, info=info, cdn_cache=False, progress=False)
cv_up.commit_info()
cv_up[:,:,:] = grouped_data[1:-1,1:-1,:].astype(np.uint8) * 255


In [212]:
from zmesh import Mesher
mesher = Mesher((512,512,336))
new_section = np.zeros((grouped_data.shape[0], grouped_data.shape[1], 1), dtype=bool)
extended_grouped_data = np.concatenate((new_section, grouped_data, new_section), axis=2)

mesher.mesh(extended_grouped_data[1:-1,1:-1,:].astype(np.uint8))
mesh = mesher.get(1, normals=True, max_error=0, reduction_factor=0, voxel_centered=False)
mesh.vertices[:,2] -= 336.0  # Undo the 1 voxel padding
mesher.erase(1)
mesher.clear()

In [213]:
with open("tissue_mesh.ply", "wb") as f:
    f.write(mesh.to_ply())

In [253]:
import trimesh
import zmesh
# import DracoPy
with open("tissue_mesh_simplified.ply", "rb") as f:
    mesh = trimesh.load_mesh(f, "ply")

m = zmesh.Mesh(mesh.vertices.astype(np.float32), mesh.faces.astype(np.uint32), mesh.vertex_normals.astype(np.float32))
# with open("tissue_mesh_simplified.drc", "wb") as f:
#     draco = DracoPy.encode(m.vertices, faces=m.faces)

from cloudfiles import CloudFiles
cf = CloudFiles("gs://dkronauer-ant-001-alignment-final/tissue_mesh")
cf.put("mesh/tissue_mesh.frag", m.to_precomputed(), compress=False, raw=True, content_type='application/octet-stream')
cf.put_json("mesh/1:0", {"fragments": ["tissue_mesh.frag"]}, compress=False)
cf.put_json("mesh/info", {"@type": "neuroglancer_legacy_mesh"})


1